In [32]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')

In [33]:
path='D:\Python\Elo\Stack'
df_train=pd.DataFrame()
print(os.listdir(path))
for name in os.listdir(path):
    if 'prediction' in name:
        df_train[name[11:-3]]=pd.read_csv(os.path.join(path,name))['target']
path='D:\Python\Elo'
df_train['target']=pd.read_csv(os.path.join(path,'main.csv'))['target']

# df_train['card_id']=pd.read_csv(os.path.join(path,'main.csv'))['card_id']
# data = pd.read_csv(os.path.join(path,'result.csv'))
# selector=['hist_month_lag_mean', 'hist_month_diff_mean',
#        'hist_fathers_day_2017_mean', 'hist_duration_min',
#        'hist_purchase_date_diff', 'hist_purchase_date_uptonow',
#        'hist_last_installments_sum_0', 'hist_purchase_amount_mean_-4',
#        'new_purchase_mean_last_3month_mean','card_id']
# df_train=df_train.merge(data[selector],how='left',on='card_id')

['prediction_LGB_6part_3.66.csv', 'prediction_LGB_7part_3.65.csv', 'prediction_LGB_7part_3.66.csv', 'prediction_LGB_8part_3.66.csv', 'prediction_LGB_full_1.55.csv', 'prediction_LGB_full_3.64544.csv', 'prediction_LGB_full_3.64573.csv', 'prediction_LGB_full_3.64753.csv', 'prediction_LGB_full_3.64904.csv', 'prediction_LGB_half_3.65.csv', 'prediction_LGB_part_1.62.csv', 'prediction_LGB_part_3.72522.csv', 'prediction_LGB_part_3.72907.csv', 'prediction_NN_3.7283.csv', 'prediction_PROB_CAT_7part_0.89.csv', 'prediction_PROB_CAT_8part_0.89.csv', 'prediction_PROB_LGB_6part_0.9.csv', 'prediction_PROB_LGB_7part_0.9.csv', 'prediction_PROB_LGB_half_0.9.csv', 'prediction_PROB_NN_0.8797.csv', 'submission_LGB_6part_3.66.csv', 'submission_LGB_7part_3.65.csv', 'submission_LGB_7part_3.66.csv', 'submission_LGB_8part_3.66.csv', 'submission_LGB_full_1.55.csv', 'submission_LGB_full_3.64544.csv', 'submission_LGB_full_3.64573.csv', 'submission_LGB_full_3.64753.csv', 'submission_LGB_full_3.64904.csv', 'submissio

In [63]:
from catboost import CatBoostRegressor, Pool
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
feature_importance_df = pd.DataFrame()
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    pool=Pool(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pool=Pool(df_train[df_train_columns].loc[val_index],df_train['target'].loc[val_index])
    num_round = 10000
    model = CatBoostRegressor(iterations=1000,learning_rate=0.01, loss_function='RMSE',early_stopping_rounds=300)
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=True)
    fold_importance_df=pd.DataFrame()
    fold_importance_df['feature'] = df_train_columns
    fold_importance_df['importance']=model.feature_importances_
    feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
np.sqrt(mean_squared_error(val_pr,df_train['target']))

0:	learn: 3.8658517	test: 3.8694275	best: 3.8694275 (0)	total: 124ms	remaining: 2m 4s
1:	learn: 3.8618426	test: 3.8657170	best: 3.8657170 (1)	total: 164ms	remaining: 1m 21s
2:	learn: 3.8578968	test: 3.8621123	best: 3.8621123 (2)	total: 201ms	remaining: 1m 6s
3:	learn: 3.8539694	test: 3.8585362	best: 3.8585362 (3)	total: 241ms	remaining: 59.9s
4:	learn: 3.8501685	test: 3.8550737	best: 3.8550737 (4)	total: 291ms	remaining: 57.8s
5:	learn: 3.8464181	test: 3.8517488	best: 3.8517488 (5)	total: 359ms	remaining: 59.5s
6:	learn: 3.8427159	test: 3.8483363	best: 3.8483363 (6)	total: 428ms	remaining: 1m
7:	learn: 3.8391018	test: 3.8450441	best: 3.8450441 (7)	total: 496ms	remaining: 1m 1s
8:	learn: 3.8356485	test: 3.8418602	best: 3.8418602 (8)	total: 540ms	remaining: 59.4s
9:	learn: 3.8321328	test: 3.8386769	best: 3.8386769 (9)	total: 578ms	remaining: 57.3s
10:	learn: 3.8286733	test: 3.8356371	best: 3.8356371 (10)	total: 623ms	remaining: 56s
11:	learn: 3.8252767	test: 3.8324526	best: 3.8324526 (11

95:	learn: 3.6774010	test: 3.7055525	best: 3.7055525 (95)	total: 3.83s	remaining: 36.1s
96:	learn: 3.6766418	test: 3.7049526	best: 3.7049526 (96)	total: 3.87s	remaining: 36s
97:	learn: 3.6758501	test: 3.7044075	best: 3.7044075 (97)	total: 3.91s	remaining: 36s
98:	learn: 3.6750505	test: 3.7038369	best: 3.7038369 (98)	total: 3.94s	remaining: 35.9s
99:	learn: 3.6742839	test: 3.7032641	best: 3.7032641 (99)	total: 3.98s	remaining: 35.8s
100:	learn: 3.6735795	test: 3.7027063	best: 3.7027063 (100)	total: 4.02s	remaining: 35.8s
101:	learn: 3.6728642	test: 3.7021581	best: 3.7021581 (101)	total: 4.05s	remaining: 35.7s
102:	learn: 3.6721790	test: 3.7015908	best: 3.7015908 (102)	total: 4.09s	remaining: 35.6s
103:	learn: 3.6714537	test: 3.7010439	best: 3.7010439 (103)	total: 4.13s	remaining: 35.6s
104:	learn: 3.6708132	test: 3.7005874	best: 3.7005874 (104)	total: 4.16s	remaining: 35.5s
105:	learn: 3.6701455	test: 3.7000467	best: 3.7000467 (105)	total: 4.22s	remaining: 35.6s
106:	learn: 3.6695116	te

188:	learn: 3.6389959	test: 3.6809891	best: 3.6809891 (188)	total: 7.39s	remaining: 31.7s
189:	learn: 3.6387803	test: 3.6809257	best: 3.6809257 (189)	total: 7.44s	remaining: 31.7s
190:	learn: 3.6385624	test: 3.6808554	best: 3.6808554 (190)	total: 7.47s	remaining: 31.7s
191:	learn: 3.6383673	test: 3.6807642	best: 3.6807642 (191)	total: 7.51s	remaining: 31.6s
192:	learn: 3.6381543	test: 3.6806659	best: 3.6806659 (192)	total: 7.54s	remaining: 31.5s
193:	learn: 3.6379782	test: 3.6805572	best: 3.6805572 (193)	total: 7.58s	remaining: 31.5s
194:	learn: 3.6377393	test: 3.6804768	best: 3.6804768 (194)	total: 7.62s	remaining: 31.5s
195:	learn: 3.6375596	test: 3.6803831	best: 3.6803831 (195)	total: 7.67s	remaining: 31.5s
196:	learn: 3.6373536	test: 3.6803374	best: 3.6803374 (196)	total: 7.71s	remaining: 31.4s
197:	learn: 3.6371822	test: 3.6802650	best: 3.6802650 (197)	total: 7.75s	remaining: 31.4s
198:	learn: 3.6369688	test: 3.6802164	best: 3.6802164 (198)	total: 7.79s	remaining: 31.3s
199:	learn

281:	learn: 3.6250378	test: 3.6777477	best: 3.6777477 (281)	total: 11.4s	remaining: 29.1s
282:	learn: 3.6249420	test: 3.6777438	best: 3.6777438 (282)	total: 11.5s	remaining: 29s
283:	learn: 3.6248745	test: 3.6777379	best: 3.6777379 (283)	total: 11.5s	remaining: 29s
284:	learn: 3.6247734	test: 3.6777235	best: 3.6777235 (284)	total: 11.5s	remaining: 29s
285:	learn: 3.6246963	test: 3.6776976	best: 3.6776976 (285)	total: 11.6s	remaining: 29s
286:	learn: 3.6245158	test: 3.6776682	best: 3.6776682 (286)	total: 11.7s	remaining: 29s
287:	learn: 3.6244128	test: 3.6776271	best: 3.6776271 (287)	total: 11.7s	remaining: 29s
288:	learn: 3.6243453	test: 3.6775990	best: 3.6775990 (288)	total: 11.8s	remaining: 29s
289:	learn: 3.6242696	test: 3.6775592	best: 3.6775592 (289)	total: 11.8s	remaining: 29s
290:	learn: 3.6241740	test: 3.6775741	best: 3.6775592 (289)	total: 11.9s	remaining: 29s
291:	learn: 3.6240972	test: 3.6775460	best: 3.6775460 (291)	total: 11.9s	remaining: 28.9s
292:	learn: 3.6240058	test: 

376:	learn: 3.6150527	test: 3.6774909	best: 3.6771911 (328)	total: 15.4s	remaining: 25.5s
377:	learn: 3.6149081	test: 3.6775351	best: 3.6771911 (328)	total: 15.4s	remaining: 25.4s
378:	learn: 3.6148227	test: 3.6775375	best: 3.6771911 (328)	total: 15.5s	remaining: 25.4s
379:	learn: 3.6147657	test: 3.6775078	best: 3.6771911 (328)	total: 15.5s	remaining: 25.3s
380:	learn: 3.6147198	test: 3.6774862	best: 3.6771911 (328)	total: 15.6s	remaining: 25.3s
381:	learn: 3.6146684	test: 3.6775014	best: 3.6771911 (328)	total: 15.6s	remaining: 25.2s
382:	learn: 3.6145818	test: 3.6775232	best: 3.6771911 (328)	total: 15.6s	remaining: 25.2s
383:	learn: 3.6144896	test: 3.6775382	best: 3.6771911 (328)	total: 15.7s	remaining: 25.2s
384:	learn: 3.6143933	test: 3.6775169	best: 3.6771911 (328)	total: 15.7s	remaining: 25.1s
385:	learn: 3.6143345	test: 3.6775005	best: 3.6771911 (328)	total: 15.8s	remaining: 25.1s
386:	learn: 3.6142651	test: 3.6774898	best: 3.6771911 (328)	total: 15.8s	remaining: 25.1s
387:	learn

...

527:	learn: 3.6048367	test: 3.6526276	best: 3.6525891 (523)	total: 18.8s	remaining: 16.8s
528:	learn: 3.6046984	test: 3.6526305	best: 3.6525891 (523)	total: 18.8s	remaining: 16.8s
529:	learn: 3.6046350	test: 3.6526345	best: 3.6525891 (523)	total: 18.8s	remaining: 16.7s
530:	learn: 3.6046080	test: 3.6526202	best: 3.6525891 (523)	total: 18.9s	remaining: 16.7s
531:	learn: 3.6045199	test: 3.6526438	best: 3.6525891 (523)	total: 18.9s	remaining: 16.6s
532:	learn: 3.6044746	test: 3.6526323	best: 3.6525891 (523)	total: 18.9s	remaining: 16.6s
533:	learn: 3.6043523	test: 3.6525865	best: 3.6525865 (533)	total: 19s	remaining: 16.6s
534:	learn: 3.6042934	test: 3.6525510	best: 3.6525510 (534)	total: 19s	remaining: 16.5s
535:	learn: 3.6042122	test: 3.6525265	best: 3.6525265 (535)	total: 19s	remaining: 16.5s
536:	learn: 3.6041346	test: 3.6525243	best: 3.6525243 (536)	total: 19.1s	remaining: 16.4s
537:	learn: 3.6040599	test: 3.6525279	best: 3.6525243 (536)	total: 19.1s	remaining: 16.4s
538:	learn: 3.60

624:	learn: 3.5959172	test: 3.6524405	best: 3.6522893 (595)	total: 22.2s	remaining: 13.3s
625:	learn: 3.5957485	test: 3.6524601	best: 3.6522893 (595)	total: 22.3s	remaining: 13.3s
626:	learn: 3.5956893	test: 3.6524562	best: 3.6522893 (595)	total: 22.3s	remaining: 13.3s
627:	learn: 3.5956130	test: 3.6525008	best: 3.6522893 (595)	total: 22.3s	remaining: 13.2s
628:	learn: 3.5955268	test: 3.6525055	best: 3.6522893 (595)	total: 22.4s	remaining: 13.2s
629:	learn: 3.5954405	test: 3.6525344	best: 3.6522893 (595)	total: 22.4s	remaining: 13.2s
630:	learn: 3.5952676	test: 3.6526516	best: 3.6522893 (595)	total: 22.4s	remaining: 13.1s
631:	learn: 3.5951397	test: 3.6526739	best: 3.6522893 (595)	total: 22.5s	remaining: 13.1s
632:	learn: 3.5950887	test: 3.6526390	best: 3.6522893 (595)	total: 22.5s	remaining: 13.1s
633:	learn: 3.5949913	test: 3.6527001	best: 3.6522893 (595)	total: 22.6s	remaining: 13s
634:	learn: 3.5948688	test: 3.6526933	best: 3.6522893 (595)	total: 22.6s	remaining: 13s
635:	learn: 3.

718:	learn: 3.5869922	test: 3.6529282	best: 3.6522893 (595)	total: 25.4s	remaining: 9.94s
719:	learn: 3.5869132	test: 3.6529028	best: 3.6522893 (595)	total: 25.5s	remaining: 9.9s
720:	learn: 3.5868190	test: 3.6529161	best: 3.6522893 (595)	total: 25.5s	remaining: 9.87s
721:	learn: 3.5866828	test: 3.6528903	best: 3.6522893 (595)	total: 25.5s	remaining: 9.83s
722:	learn: 3.5866453	test: 3.6528916	best: 3.6522893 (595)	total: 25.6s	remaining: 9.79s
723:	learn: 3.5865574	test: 3.6529059	best: 3.6522893 (595)	total: 25.6s	remaining: 9.76s
724:	learn: 3.5864093	test: 3.6529199	best: 3.6522893 (595)	total: 25.6s	remaining: 9.72s
725:	learn: 3.5863286	test: 3.6529525	best: 3.6522893 (595)	total: 25.7s	remaining: 9.69s
726:	learn: 3.5861739	test: 3.6530143	best: 3.6522893 (595)	total: 25.7s	remaining: 9.65s
727:	learn: 3.5861099	test: 3.6529841	best: 3.6522893 (595)	total: 25.7s	remaining: 9.61s
728:	learn: 3.5859211	test: 3.6529780	best: 3.6522893 (595)	total: 25.8s	remaining: 9.58s
729:	learn:

812:	learn: 3.5783201	test: 3.6537026	best: 3.6522893 (595)	total: 28.5s	remaining: 6.57s
813:	learn: 3.5782804	test: 3.6537101	best: 3.6522893 (595)	total: 28.6s	remaining: 6.53s
814:	learn: 3.5782284	test: 3.6536980	best: 3.6522893 (595)	total: 28.6s	remaining: 6.5s
815:	learn: 3.5781624	test: 3.6536952	best: 3.6522893 (595)	total: 28.7s	remaining: 6.46s
816:	learn: 3.5781196	test: 3.6536844	best: 3.6522893 (595)	total: 28.7s	remaining: 6.43s
817:	learn: 3.5780919	test: 3.6536834	best: 3.6522893 (595)	total: 28.7s	remaining: 6.39s
818:	learn: 3.5780150	test: 3.6536874	best: 3.6522893 (595)	total: 28.8s	remaining: 6.36s
819:	learn: 3.5778749	test: 3.6537048	best: 3.6522893 (595)	total: 28.8s	remaining: 6.32s
820:	learn: 3.5777549	test: 3.6537184	best: 3.6522893 (595)	total: 28.8s	remaining: 6.29s
821:	learn: 3.5776685	test: 3.6537448	best: 3.6522893 (595)	total: 28.9s	remaining: 6.25s
822:	learn: 3.5776407	test: 3.6537456	best: 3.6522893 (595)	total: 28.9s	remaining: 6.22s
823:	learn:

3.646357535267267

In [53]:
cols=(feature_importance_df.groupby('feature').mean().sort_values(by="importance", ascending=False))
cols.reset_index(drop=False,inplace=True)
plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
           data=cols)
plt.title('LightGBM Features (avg over folds)')
plt.show()

DataError: No numeric types to aggregate

In [48]:
from sklearn.feature_selection import RFECV
model = Ridge(alpha=60)
selector = RFECV(model, step=1, cv=5)
selector=selector.fit(df_train[df_train_columns],df_train['target'])

In [55]:
from catboost import CatBoostRegressor, Pool
from sklearn.linear_model import Ridge
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
oblig=['LGB_7part_3.65.', 'LGB_full_3.64544.', 'LGB_full_3.64573.',
       'LGB_full_3.64753.', 'LGB_part_3.72522.', 'LGB_part_3.72907.',
       'NN_3.7283.', 'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.',
       'PROB_LGB_6part_0.9.', 'PROB_LGB_7part_0.9.', 'PROB_LGB_half_0.9.',
       'PROB_NN_0.8797.']
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id'] and c in oblig]
df_train=df_train.replace([np.inf, -np.inf], np.nan)
df_train=df_train.fillna(0)
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    num_round = 10000
    model = Ridge(alpha=60)
    model.fit(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
np.sqrt(mean_squared_error(val_pr,df_train['target']))

3.6426215434279285

In [77]:
# from sklearn.feature_selection import RFECV
# model = Ridge()
# selector = RFECV(model, step=1, cv=5)
# selector=selector.fit(df_train[df_train_columns],df_train['target'])
from catboost import CatBoostRegressor, Pool
from sklearn.linear_model import Ridge
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
oblig=['LGB_full_3.64544.', 'LGB_full_3.64573.', 'LGB_full_3.64753.',
       'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.', 'PROB_LGB_half_0.9.']
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
df_train=df_train.replace([np.inf, -np.inf], np.nan)
df_train=df_train.fillna(0)
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    pool=Pool(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pool=Pool(df_train[df_train_columns].loc[val_index],df_train['target'].loc[val_index])
    num_round = 10000
    model = Ridge()
    model.fit(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
np.sqrt(mean_squared_error(val_pr,df_train['target']))

Index(['LGB_full_3.64544.', 'LGB_full_3.64573.', 'LGB_full_3.64753.',
       'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.', 'PROB_LGB_half_0.9.'],
      dtype='object')

In [69]:
# from sklearn.model_selection import train_test_split
# train,test=train_test_split(df_train[['LGB_full_3.64544.','NN_3.7283.','LGB_7part_3.65.','LGB_6part_3.66.','LGB_half_3.65.','target']],test_size=0.2)
# def weight_pred(a1,a2,a3,a4,a5):
#     train['pr']=train.iloc[:,0]*a1+train.iloc[:,1]*a2+train.iloc[:,2]*a3+train.iloc[:,3]*a4+train.iloc[:,4]*a5
#     return 1-np.sqrt(mean_squared_error(train['pr'],train['target']))


# from bayes_opt import BayesianOptimization
# optimizer = BayesianOptimization(weight_pred,{'a1':(0,1),'a2':(0,1),'a3':(0,1),'a4':(0,1),'a5':(0,1)})
# optimizer.maximize(init_points=3,
#     n_iter=40)
np.sqrt(mean_squared_error(np.mean([val_pr,pr_cat],axis=0),df_train['target']))

3.645623590006504

In [66]:
param = {'objective': 'regression_l2', 
    'metric': 'rmse',
            'boosting_type': 'gbdt', 
            'n_jobs': 4, 'max_depth': 2, 
            'n_estimators': 5900, 
            'subsample_freq': 2, 
            'min_data_per_group': 100, 
            'max_cat_to_onehot': 4, 
            'cat_l2': 10.0, 
            'cat_smooth': 10.0, 
            'max_cat_threshold': 32, 
            'metric_freq': 10, 
            'verbosity': -1, 
            'colsample_bytree': 0.5, 
            'learning_rate': 0.0061033234451294376, 
            'min_child_samples': 195, 
            'min_child_weight': 49.0, 
            'min_split_gain': 1e-06, 
            'num_leaves': 24, 
            'reg_alpha': 15, 
            'reg_lambda': 186, 
            'subsample': 0.9}
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
skf=StratifiedKFold(5,shuffle=True,random_state=100)
val_pr=np.zeros(len(df_train))
feature_importance_df = pd.DataFrame()
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    train_data=lgb.Dataset(df_train[df_train_columns].loc[train_index],label=df_train['target'].loc[train_index])
    val_data=lgb.Dataset(df_train[df_train_columns].loc[val_index],label=df_train['target'].loc[val_index])
    num_round = 10000
    model=lgb.train(param,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=100,early_stopping_rounds=400)
    fold_importance_df=pd.DataFrame()
    fold_importance_df['feature'] = df_train_columns
    fold_importance_df['importance']=model.feature_importance
    feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index],num_iteration=model.best_iteration)
np.sqrt(mean_squared_error(val_pr,df_train['target']))

Training until validation scores don't improve for 400 rounds.
[100]	training's rmse: 3.72336	valid_1's rmse: 3.74161
[200]	training's rmse: 3.67351	valid_1's rmse: 3.69677
[300]	training's rmse: 3.65212	valid_1's rmse: 3.67868
[400]	training's rmse: 3.64256	valid_1's rmse: 3.67111
[500]	training's rmse: 3.63806	valid_1's rmse: 3.6677
[600]	training's rmse: 3.63568	valid_1's rmse: 3.66612
[700]	training's rmse: 3.63425	valid_1's rmse: 3.6653
[800]	training's rmse: 3.63322	valid_1's rmse: 3.66484
[900]	training's rmse: 3.63234	valid_1's rmse: 3.66467
[1000]	training's rmse: 3.63147	valid_1's rmse: 3.66442
[1100]	training's rmse: 3.63069	valid_1's rmse: 3.66426
[1200]	training's rmse: 3.62992	valid_1's rmse: 3.66422
[1300]	training's rmse: 3.62915	valid_1's rmse: 3.6642
[1400]	training's rmse: 3.62845	valid_1's rmse: 3.66419
[1500]	training's rmse: 3.62779	valid_1's rmse: 3.66423
[1600]	training's rmse: 3.6271	valid_1's rmse: 3.66422
Early stopping, best iteration is:
[1275]	training's r

3.6461458750600344

In [34]:
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
def optim_lgb(min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight):
    def run_lgb(train_data,val_data,val_x,min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight):
        num_round = 10000
        params = {
                    "objective" : "regression_l2",
                    "metric" : "rmse", 
                    "boosting": "gbdt",
                    'n_jobs': 6,
                   'learning_rate': 0.008,
                    'max_cat_to_onehot': 4, 
                     'min_child_samples': int(min_child),
                    'cat_smooth': 10.0,
                    'n_estimators': int(n_estimator),
                     'max_depth':int(round(max_depth)),
                     'num_leaves':int(num_leaves),
                     'lambda_l1':max(lambda_l1, 0),
                     'lambda_l2':max(lambda_l2, 0),
                     'min_split_gain':min_split_gain,
                     'min_child_weight':min_child_weight
                }
        model=lgb.train(params,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=False,early_stopping_rounds=400)

        pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    #    return pred_val
        return  pred_val
    rskf=StratifiedKFold(5,shuffle=True,random_state=315)
    val_pr=np.zeros(len(df_train))
    for train_index,val_index in rskf.split(df_train,df_train['outliers'].values):
        train_data=lgb.Dataset(df_train[df_train_columns].loc[train_index],label=df_train['target'].loc[train_index])
        val_data=lgb.Dataset(df_train[df_train_columns].loc[val_index],label=df_train['target'].loc[val_index])
        val_pr[val_index]=run_lgb(train_data,val_data,df_train[df_train_columns].loc[val_index],min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight)
    return 1-np.sqrt(mean_squared_error(val_pr,df_train['target']))
    
from bayes_opt import BayesianOptimization
optimizer = BayesianOptimization(optim_lgb,{'min_child':(5,200),'n_estimator':(50,7000),'max_depth':(-1,20),'num_leaves':(10,50),'lambda_l1':(0,200),'lambda_l2':(0,200),
'min_split_gain':(0.001, 0.1),'min_child_weight':(2,50)}
    
)
optimizer.maximize(init_points=3,
    n_iter=30)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_child | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -2.651    |  51.89    |  23.21    |  18.44    |  73.71    |  47.35    |  0.06746  |  1.403e+0 |  39.69    |
|  2        | -2.647    |  180.5    |  13.73    |  1.93     |  25.26    |  6.935    |  0.06764  |  3.531e+0 |  41.85    |
|  3        | -2.649    |  191.1    |  81.7     |  16.49    |  56.18    |  6.782    |  0.05252  |  1.759e+0 |  28.0     |
|  4        | -2.647    |  100.8    |  196.5    |  4.09     |  199.8    |  38.24    |  0.09917  |  6.972e+0 |  31.91    |
|  5        | -2.647    |  181.4    |  2.822    |  1.621    |  12.44    |  32.73    |  0.02278  |  6.984e+0 |  39.46    |
|  6        | -2.648    |  1.597    |  196.7    |  11.3     |  14.59    |  16.85    |  0.02463  |  4.945e+0 |  11.48    |
|  7        | -2.647    

In [51]:
df_train[df_train_columns].columns[selector.support_]

Index(['LGB_7part_3.65.', 'LGB_full_3.64544.', 'LGB_full_3.64573.',
       'LGB_full_3.64753.', 'LGB_part_3.72522.', 'LGB_part_3.72907.',
       'NN_3.7283.', 'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.',
       'PROB_LGB_6part_0.9.', 'PROB_LGB_7part_0.9.', 'PROB_LGB_half_0.9.',
       'PROB_NN_0.8797.'],
      dtype='object')

In [39]:
# from sklearn.feature_selection import RFECV
# model = Ridge()
# selector = RFECV(model, step=1, cv=5)
# selector=selector.fit(df_train[df_train_columns],df_train['target'])
from sklearn.ensemble import RandomForestRegressor
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
oblig=['LGB_full_3.64544.', 'LGB_full_3.64573.', 'LGB_full_3.64753.',
       'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.', 'PROB_LGB_half_0.9.']
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
df_train=df_train.replace([np.inf, -np.inf], np.nan)
df_train=df_train.fillna(0)
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    num_round = 10000
    model = RandomForestRegressor(n_estimators=2000,n_jobs=-1)
    model.fit(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
np.sqrt(mean_squared_error(val_pr,df_train['target']))

KeyboardInterrupt: 

In [45]:
from sklearn.ensemble import ExtraTreesRegressor 
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
oblig=['LGB_full_3.64544.', 'LGB_full_3.64573.', 'LGB_full_3.64753.',
       'PROB_CAT_7part_0.89.', 'PROB_CAT_8part_0.89.', 'PROB_LGB_half_0.9.']
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
df_train=df_train.replace([np.inf, -np.inf], np.nan)
df_train=df_train.fillna(0)
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    num_round = 10000
    model = ExtraTreesRegressor()
    model.fit(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
np.sqrt(mean_squared_error(val_pr,df_train['target']))

3.891955758133225

In [54]:
result=pd.DataFrame(columns=['alpha','rmse'])
for i in [0.1,0.5,0.8,1,3,6,12,20,40,60,100,300,500,1000,3000]:
    skf=StratifiedKFold(5,shuffle=True)
    val_pr=np.zeros(len(df_train))
    for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
        num_round = 10000
        model = Ridge(alpha=i)
        model.fit(df_train[df_train_columns].loc[train_index],df_train['target'].loc[train_index])
        val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index])
    result.loc[len(result)]=[i,np.sqrt(mean_squared_error(val_pr,df_train['target']))]
result

,alpha,rmse
0,0.1,3.644017
1,0.5,3.643678
2,0.8,3.642965
3,1.0,3.643142
4,3.0,3.642728
5,6.0,3.642625
6,12.0,3.643088
7,20.0,3.642545
8,40.0,3.642496
9,60.0,3.642780
